<a href="https://colab.research.google.com/github/mehri-satari/Data-Mining-Course-Project/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [ ]:
1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


SyntaxError: unmatched ')' (ipython-input-565355372.py, line 1)

### 2) Common Imports & Polite Headers

In [ ]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


In [ ]:
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")

Dependencies installed.


In [ ]:
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0 Safari/537.36")
}

def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text

def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(x) for x in tup if str(x) != "nan"]).strip()
            for tup in df.columns.values
        ]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df

print("Common helpers loaded.")

Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [ ]:
url = "https://www.iban.com/country-codes"
html = fetch_html(url)
df_raw = pd.read_html(html)[0]

def q1_clean(df):
    df.columns = [c.strip() for c in df.columns]
    for col in ('Alpha-2 code', 'Alpha-3 code'):
        df[col] = df[col].astype(str).str.strip().str.upper()
    df['Numeric'] = pd.to_numeric(df['Numeric'], errors='coerce').astype('Int64')
    df = df.dropna(subset=['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric'])
    df['Country'] = df['Country'].astype(str).str.strip()
    return df

df_clean = q1_clean(df_raw)
df_clean.to_csv("data_q1.csv", index=False)
top15 = df_clean.sort_values('Numeric', ascending=False).head(15)
print(top15)

                                               Country Alpha-2 code  \
247                                             Zambia           ZM   
246                                              Yemen           YE   
192                                              Samoa           WS   
244                                  Wallis and Futuna           WF   
240                 Venezuela (Bolivarian Republic of)           VE   
238                                         Uzbekistan           UZ   
237                                            Uruguay           UY   
35                                        Burkina Faso           BF   
243                              Virgin Islands (U.S.)           VI   
236                     United States of America (the)           US   
219                       Tanzania, United Republic of           TZ   
108                                        Isle of Man           IM   
113                                             Jersey           JE   
92    

/tmp/ipython-input-2866527099.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_raw = pd.read_html(html)[0]


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [ ]:
def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "lxml")
    items = []
    for row in soup.select("tr.athing"):
        rank_tag = row.select_one("span.rank")
        rank = rank_tag.text.strip().replace('.', '') if rank_tag else ""
        title_tag = row.select_one("a.storylink") or row.select_one("span.titleline > a")
        title = title_tag.text.strip() if title_tag else ""
        link = title_tag['href'] if title_tag and 'href' in title_tag.attrs else ""
        subtext = row.find_next_sibling("tr").select_one(".subtext")
        points_tag = subtext.select_one("span.score") if subtext else None
        points = points_tag.text.strip().replace(" points", "") if points_tag else ""
        user_tag = subtext.select_one("a.hnuser") if subtext else None
        user = user_tag.text.strip() if user_tag else ""
        comments_tag = subtext.find_all("a")[-1] if subtext and subtext.find_all("a") else None
        if comments_tag and "comment" in comments_tag.text:
            comments = comments_tag.text.strip().split()[0]
        else:
            comments = ""
        items.append({
            "rank": rank,
            "title": title,
            "link": link,
            "points": points,
            "comments": comments,
            "user": user
        })
    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    for col in ["rank", "points", "comments"]:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(r"[^\d]", "", regex=True), errors="coerce").fillna(0).astype(int)
    df["title"] = df["title"].fillna("")
    df["link"] = df["link"].fillna("")
    df["user"] = df["user"].fillna("")
    return df

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values("points", ascending=False).head(top)

In [ ]:
url = "https://news.ycombinator.com/"
html = fetch_html(url)
df = q2_parse_items(html)
df = q2_clean(df)
df.to_csv("data_q2.csv", index=False)
print(q2_sort_top(df, 15))

    rank                                              title  \
15    16            FBI tries to unmask owner of archive.is   
4      5  Kimi K2 Thinking, a SOTA open-source trillion-...   
1      2                          You should write an agent   
2      3           Two billion email addresses were exposed   
10    11  Open Source Implementation of Apple's Private ...   
7      8  Show HN: I scraped 3B Goodreads reviews to tra...   
23    24    Mathematical exploration and discovery at scale   
11    12                           Swift on FreeBSD Preview   
6      7                              Game design is simple   
16    17                            Eating stinging nettles   
17    18  I analyzed the lineups at the most popular nig...   
8      9  Analysis indicates that the universe’s expansi...   
14    15             LLMs encode how difficult problems are   
26    27                            The Parallel Search API   
3      4  Scientists find ways to boost memory in aging